In [1]:
# reuse the Image classification model
import torch
import os
import mlflow
import flash
from flash.core.data.utils import download_data
from flash.image import ImageClassificationData, ImageClassifier


/Users/phamthanhtu/opt/miniconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:252: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  warnings.warn(


In [2]:
# Setup the mlflow experiment and AWS access
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"

In [11]:
datamodule = ImageClassificationData.from_folders(
    train_folder="../data-2/hymenoptera_data/train/",
    val_folder="../data-2/hymenoptera_data/val/",
    test_folder="../data-2/hymenoptera_data/test/",
    batch_size=2,
    transform_kwargs={"image_size": (196, 196), "mean": (0.485, 0.456, 0.406), "std": (0.229, 0.224, 0.225)},
)

# 2. Build the task
model = ImageClassifier(backbone="resnet18", labels=datamodule.labels)
trainer = flash.Trainer(max_epochs=1, gpus=torch.cuda.device_count())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [4]:
# set experiment
EXPERIMENT_NAME = "dl_model"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
print("experiment_id:", experiment.experiment_id)

experiment_id: 1


In [ ]:
mlflow.pytorch.autolog()

with mlflow.start_run(experiment_id=experiment.experiment_id, run_name="image-classifier") as dl_model_tracking_run:
    trainer.finetune(model, datamodule=datamodule, strategy="freeze")
    trainer.test()

In [13]:
run_id = dl_model_tracking_run.info.run_id
print(run_id)
print(mlflow.get_run(run_id).info.lifecycle_stage)

12a33fe9a1324881825325148016a29b
active


In [7]:
# use the run_id to construct a logged_model URI.
logged_model = f'runs:/{run_id}/model'

In [8]:
# Load model 
loaded_model = mlflow.pytorch.load_model(logged_model)

In [ ]:
# predict
datamodule = ImageClassificationData.from_files(
    predict_files=[
        "../data-2/hymenoptera_data/predict/153783656_85f9c3ac70.jpg" ],
    batch_size=1,
)
predictions = trainer.predict(loaded_model, datamodule=datamodule)
print(predictions)

In [9]:
# register model
model_register = mlflow.register_model(logged_model, 'image-classifier')
print(model_register.name)
print(model_register.version)

Successfully registered model 'image-classifier'.
2022/10/18 10:36:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: image-classifier, version 1


image-classifier
1


Created version '1' of model 'image-classifier'.
